In [8]:

import numpy as np

import matplotlib.pyplot as plt
import tensorflow as tf
from collections import Counter
import random
import bisect

In [9]:
link = 'https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz'
path = tf.keras.utils.get_file('mnist.npz', link)    
DATASET = np.load(path)
x_train, y_train = DATASET['x_train'], DATASET['y_train']
x_test, y_test = DATASET['x_test'], DATASET['y_test']

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


Use https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz MNIST
dataset for this question and select two digits - 0 and 1. Label them as -1 and
1. In this exercise you will be implementing AdaBoost.M1. Perform following
tasks.

In [10]:
train_selective = np.isin(y_train , [0,1]) 
test_selective = np.isin(y_test , [0,1])

x_train ,y_train = x_train[train_selective], y_train[train_selective]
x_test , y_test = x_test[test_selective], y_test[test_selective]


print(x_train.shape)
print(y_train.shape)
# print(y_train[3])

y_train_new = np.where(y_train == 0, -1, 1)
y_train = y_train_new 

print(y_train.shape)

(12665, 28, 28)
(12665,)
(12665,)


Divide the train set into train and val set. Keep 1000 samples from each
class for val. Note val should be used to evaluate the performance of the
classifier. Must not be used in obtaining PCA matrix.

In [11]:
x_train_set = [] 
x_val_set = []
y_train_set = []
y_val_set = []

total_taken = [0]*(2) 
for i in range(len(x_train)):
  if (y_train[i] == -1 and (total_taken[0] < 1000) ):
    x_val_set.append(x_train[i]) 
    y_val_set.append(y_train[i])
    total_taken[0] += 1 
  elif (y_train[i] == 1 and (total_taken[1] < 1000)):
    x_val_set.append(x_train[i]) 
    y_val_set.append(y_train[i])
    total_taken[1] += 1
  else:
    x_train_set.append(x_train[i])
    y_train_set.append(y_train[i])

x_train_set = np.array(x_train_set)
x_val_set = np.array(x_val_set)
y_train_set = np.array(y_train_set)
y_val_set = np.array(y_val_set)
  
print(x_train_set.shape)
print(x_val_set.shape)
print(y_train_set.shape)
print(y_val_set.shape)

# thus we have created a val set of 2000 samples with 1000 samples of each class each ie . -1 and 1 
 


(10665, 28, 28)
(2000, 28, 28)
(10665,)
(2000,)


In [12]:
# Now lets reduce the dimensions of the train set .
x_train_set = x_train_set.reshape(x_train_set.shape[0] , -1) 
x_val_set = x_val_set.reshape(x_val_set.shape[0] ,  -1)

x_train_set = x_train_set.T
x_val_set = x_val_set.T


print(x_train_set.shape)
print(x_val_set.shape)

# changing the dimension of the train set first 
X = x_train_set
mean_X = np.mean(X, axis=1, keepdims=True)
print(mean_X.shape)

X_centered = X - mean_X

S = (X_centered @ np.transpose(X_centered)) / (X_centered.shape[1] - 1) 

eigenvalues, eigenvectors = np.linalg.eig(S)
sorted_indices = np.argsort(eigenvalues)[::-1]
eigenvalues = eigenvalues[sorted_indices]
eigenvectors = eigenvectors[:, sorted_indices]

U = eigenvectors
p = 5

x_train_set = np.transpose(U[:, :p]) @ X_centered


print(x_train_set.shape)
x_train_set = x_train_set.T 

# Now we will change the dimensions of the val test also  . 

X = x_val_set
mean_X = np.mean(X, axis=1, keepdims=True)
print(mean_X.shape)

X_centered = X - mean_X

S = (X_centered @ np.transpose(X_centered)) / (X_centered.shape[1] - 1) 

eigenvalues, eigenvectors = np.linalg.eigh(S)
sorted_indices = np.argsort(eigenvalues)[::-1]
eigenvalues = eigenvalues[sorted_indices]
eigenvectors = eigenvectors[:, sorted_indices]

U = eigenvectors
p = 5

x_val_set = np.transpose(U[:, :p]) @ X_centered


print(x_val_set.shape)
x_val_set = x_val_set.T 

unique_Values = [np.unique(x_train_set[:, i]) for i in range(p)]

 


(784, 10665)
(784, 2000)
(784, 1)
(5, 10665)
(784, 1)
(5, 2000)


In [13]:
print(unique_Values[0])

[-2330.82976086+0.j -2327.20344266+0.j -2290.15016654+0.j ...
  1128.97522928+0.j  1138.47192104+0.j  1139.9026577 +0.j]


Now learn a decision tree using the train set. You need to grow a deci-
sion stump. For each dimension, find the unique values and sort them

in ascending order. The splits to be evaluated will be midpoint of two
consecutive unique values. Find the best split by minimizing weighted
miss-classification error. Denote this as h1(x). Note as we are dealing
with real numbers, each value may be unique. So just sorting them and
taking midpoint of consecutive values may also result in similar tree.

In [14]:
weights = [1/(len(x_train_set))]*(len(x_train_set))
# storing the information of the classifiers in arrays the split value and the dimension chosen for the decision stumps 
dimension_chosen_array = []  ; split_chosen_array = [] ; left_val_array = [] ; right_val_array = [] ; alpha_value = [] ; losses_array = []
# print(weights)

def calculate_loss_function(x_train_set, y_train_set, weights, dimension, split_value, sorted_indices, indx):
    total_weights_loss = 0
    total_weights = np.sum(weights)
    wrongly_classified_points = []

    x_train_set = np.array(x_train_set)
    y_train_set = np.array(y_train_set)
    weights = np.array(weights)
    sorted_indices = np.array(sorted_indices)

    # Index-based split
    left_indices = np.arange(indx)
    right_indices = np.arange(indx, len(x_train_set))

    split_left = weights[left_indices]
    split_right = weights[right_indices]
    classes_left = y_train_set[left_indices]
    classes_right = y_train_set[right_indices]
    indices_left = sorted_indices[left_indices]
    indices_right = sorted_indices[right_indices]

    count1_left = np.sum(classes_left == -1)
    count2_left = np.sum(classes_left == 1)
    count1_right = np.sum(classes_right == -1)
    count2_right = np.sum(classes_right == 1)

    decided_class_left = -1 if count1_left > count2_left else 1
    decided_class_right = -1 if count1_right > count2_right else 1

    wrong_left = classes_left != decided_class_left
    wrong_right = classes_right != decided_class_right

    total_weights_loss += np.sum(split_left[wrong_left])
    total_weights_loss += np.sum(split_right[wrong_right])

    wrongly_classified_points = np.concatenate((indices_left[wrong_left], indices_right[wrong_right]))

    ans = total_weights_loss / total_weights
    return ans, wrongly_classified_points, decided_class_left, decided_class_right


  




  
def minimum_each_dimension(x_train_set, y_train_set , weights , dimension):
  unique_values = unique_Values[dimension]
  
  # unique_values = np.random.choice(unique_values , size=1000 , replace=False)
  
#   unique_values = np.unique(x_train_set[:, dimension])
#   # for x in x_train_set:
#   #   if (x[dimension] not in unique_values):
#   #     unique_values.append(x[dimension])
  
#   unique_values = sorted(unique_values) 
  sorted_indices = np.argsort(x_train_set[:, dimension])
  # print(sorted_indices)
  x_train_set_sorted = x_train_set[sorted_indices]
  y_train_set_sorted = y_train_set[sorted_indices]
  weights = np.array(weights)
  weights_sorted = weights[sorted_indices]

  weights = list(weights)
  loss_val = 1.1  
  split_val = 0   
  wrongly_classified_points = []
  decided_class_left = 0 
  decided_class_right = 0 
  
  midpoints = (unique_values[1:] + unique_values[:-1]) /2 
  for i in range(len(midpoints) ):
    split_value = midpoints[i]
    # print("h1 : "+ str(split_value))
    loss_at_split , wrongly_classified_points_new , decided_class_left_new , decided_class_right_new = calculate_loss_function(x_train_set_sorted , y_train_set_sorted , weights_sorted , dimension , split_value ,sorted_indices , i)

    if (loss_val > loss_at_split):
      split_val = split_value 
      loss_val = loss_at_split  
      wrongly_classified_points = wrongly_classified_points_new 
      decided_class_left = decided_class_left_new  
      decided_class_right = decided_class_right_new 
  
  return loss_val , split_val , wrongly_classified_points , decided_class_left , decided_class_right

  
def best_dimension_and_split(x_train_set, y_train_set , weights):
  loss_val = 1.1 ; split_val = 0 ; selected_dimension = -1 ; wrongly_classified_points = [] ; decided_class_left =0 ; decided_class_right = 0
  # print(len(x_train_set))
  for x in range(len(x_train_set[1])):
    
    loss_value , split_value ,wrongly_classified_points_new , decided_class_left_new ,decided_class_right_new = minimum_each_dimension(x_train_set  , y_train_set , weights , x) 
    if (loss_val > loss_value):
      loss_val = loss_value 
      split_val = split_value 
      selected_dimension = x
      wrongly_classified_points = wrongly_classified_points_new
      decided_class_left = decided_class_left_new 
      decided_class_right = decided_class_right_new 
    # print(loss_value , split_value , wrongly_classified_points) 

  # print(len(wrongly_classified_points))
  return selected_dimension , split_val , loss_val , wrongly_classified_points , decided_class_left , decided_class_right

def find_accuracy(indx):
  ans = 0  
  for i in range(len(x_val_set)):
    count1 = 0 
    for j in range(0 , indx+1):
      if (x_val_set[i][dimension_chosen_array[j]] < split_chosen_array[j]):
        count1 += (alpha_value[j] * left_val_array[j]) 
      else:
        count1 += (alpha_value[j] * right_val_array[j]) 
    
    if (np.sign(count1) == y_val_set[i]):
      ans +=1 
  
  return ans/(len(x_val_set))

# now let me create the h1(x) from the information provided by the above functions , it is just conditionals . 
def create_classifiers(x_train_set , y_train_set, num_of_classifiers):

  for num in range(0 , num_of_classifiers):
    selected_dimension , split_val , loss_val , wrongly_classified_points , decided_class_left , decided_class_right = best_dimension_and_split(x_train_set , y_train_set , weights) 
    dimension_chosen_array.append(selected_dimension)
    split_chosen_array.append(split_val)
    left_val_array.append(decided_class_left) 
    right_val_array.append(decided_class_right)
    print(loss_val)
    losses_array.append(loss_val)
    alpha = np.log((1-loss_val) / (loss_val)) 
    alpha_value.append(alpha) 
    print(wrongly_classified_points)
    for i in wrongly_classified_points:
      weights[i] = weights[i] * (1-loss_val) / (loss_val)
    
    # print(dimension_chosen_array)
    # print(split_chosen_array)
    # print(alpha)  
    print(losses_array)
    # print(left_val_array)
    # print(right_val_array)
    # print(weights[11])
    print("this is the accuracy " + str(find_accuracy(num)))

# print(len(x_train_set))
# print(minimum_each_dimension(x_train_set , y_train_set , weights , 0))
# print(x_train_set.shape)
# print(best_dimension_and_split(x_train_set , y_train_set , weights))
(create_classifiers(x_train_set , y_train_set , 300))
# def find_best_split(x_train_set , y_train_set , weights):

0.004688232536333803
[ 8745  1740  4235  3613  4438   307  4626   333  3118  3194  9110  2337
  4820  8731    11  1225  2277  2611  4263  3629  5827  5236  4409 10594
  7092   377  8399  3615  5657  9613  5418  9434  5383 10384   703  6609
  9076  6391  2607  8658  7648  6184  3915 10392  1419  3342  6546  6022
  2495  4030]
[0.004688232536333803]
this is the accuracy 0.995
0.23610927932171452
[ 8745  1740  4235  3613  4438   307  4626   333  3118  3194  9110  2337
  4820  8731    11  1225  2277  2611  4263  3629  5827  5236  1650  1381
  1240  5348  4503  4968  5054  5474  6380  4654  2409  2697  6442  8692
  5998  7326  2225  6710  3490  4182  5795  9815  4643  1133  8926   823
  9707  6635  8903  9183  7773  2758  9876  6024  3512  1202  1995  2097
  4794  1971  5876 10536  5370  2806  5107  6770    49  1349  1495  9545
  6901  4046  5810  7662  5028  3247  7418  3444  5905  2919  8500 10273
  3548  9255   769  2432  1634  9306  1366   107  9160  7972  3466  8048
   975  2799  4750 

• Compute α1 and update weights.